In [1]:
import numpy as np
import pandas as pd
import requests
from datetime import datetime
import json

#Set Display options
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 50)
pd.set_option('display.width' , 1000)

In [2]:
# Variables
season = "2020-21"

In [3]:
all_players = pd.read_csv (r'./data/' + season + '/Cleaned_data/all_players.csv', parse_dates= ["news_added"])
all_teams = pd.read_csv (r'./data/' + season + '/Cleaned_data/all_teams.csv')
all_events = pd.read_csv (r'./data/' + season + '/Cleaned_data/all_events.csv', parse_dates= ["deadline_time"])
player_types = pd.read_csv(r'./data/' + season + '/Cleaned_data/player_types.csv')
game_phases = pd.read_csv(r'./data/' + season + '/Cleaned_data/game_phases.csv')

player_season_history = pd.read_csv(r'./data/' + season + '/Cleaned_data/player_season_history.csv')
player_gameweek_history = pd.read_csv(r'./data/' + season + '/Cleaned_data/player_gameweek_history.csv' , parse_dates = ['kickoff_time'])
player_future_fixture = pd.read_csv(r'./data/' + season + '/Cleaned_data/player_future_fixture.csv', parse_dates = ["kickoff_time"])




In [4]:
all_teams.head()

,team_code,draw,team_form,team_id,loss,team_name,played,points,position,team_short_name,team_strength,team_division,unavailable,win,strength_overall_home,strength_overall_away,strength_attack_home,strength_attack_away,strength_defence_home,strength_defence_away,pulse_id
0,3,0,NaN,1,0,Arsenal,0,0,0,ARS,4,NaN,False,0,1240,1250,1160,1210,1190,1230,1
1,7,0,NaN,2,0,Aston Villa,0,0,0,AVL,3,NaN,False,0,1130,1150,1140,1170,1120,1160,2
2,36,0,NaN,3,0,Brighton,0,0,0,BHA,3,NaN,False,0,1100,1120,1110,1130,1100,1120,131
3,90,0,NaN,4,0,Burnley,0,0,0,BUR,3,NaN,False,0,1060,1100,1130,1150,1010,1020,43
4,8,0,NaN,5,0,Chelsea,0,0,0,CHE,4,NaN,False,0,1250,1280,1180,1250,1230,1260,4


In [5]:
all_players.head()

,chance_of_playing_next_round,chance_of_playing_this_round,player_code,cost_change_event,cost_change_event_fall,cost_change_start,cost_change_start_fall,dreamteam_count,player_type_id,ep_next,ep_this,event_points,form,player_id,in_dreamteam,news,news_added,now_cost,photo,PPG,selected_by_percent,status,team_id,team_code,total_points,...,red_cards,saves,bonus,bps,influence,creativity,threat,ict_index,influence_rank,influence_rank_type,creativity_rank,creativity_rank_type,threat_rank,threat_rank_type,ict_index_rank,ict_index_rank_type,corners_and_indirect_freekicks_order,corners_and_indirect_freekicks_text,direct_freekicks_order,direct_freekicks_text,penalties_order,penalties_text,PP90,PPMM,VAPM
0,NaN,NaN,37605,0,0,-2,2,0,3,-0.5,0.5,0,0.0,1,False,NaN,NaT,68,37605.jpg,0.0,0.8,a,1,3,0,...,0,0,0,0,0.0,0.0,0.0,0.0,570,233,571,233,570,232,571,233,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,-0.294118
1,100.0,75.0,39476,0,0,-1,1,0,2,-0.5,0.4,0,0.0,2,False,NaN,2020-09-11 11:00:08.600094,49,39476.jpg,0.0,0.1,a,1,3,0,...,0,0,0,0,0.0,0.0,0.0,0.0,487,189,479,189,452,178,491,189,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,-0.408163
2,100.0,100.0,41270,0,0,0,0,0,2,0.8,1.8,2,1.3,3,False,NaN,2020-09-10 14:30:11.632386,55,41270.jpg,1.3,1.0,a,1,3,4,...,0,0,0,27,43.2,4.0,0.0,4.7,164,64,255,81,539,208,257,86,NaN,NaN,4.0,NaN,NaN,NaN,1.98895,0.236364,-0.127273
3,75.0,NaN,54694,0,0,-1,1,0,3,1.9,3.5,2,3.0,4,False,Ankle injury - 75% chance of playing,2020-10-08 09:00:12.305506,119,54694.jpg,4.0,22.6,d,1,3,16,...,0,0,0,54,78.2,57.2,103.0,23.8,69,26,60,44,30,13,41,22,NaN,NaN,6.0,NaN,1.0,NaN,4.00000,0.336134,0.168067
4,100.0,100.0,58822,0,0,-2,2,0,2,-0.5,0.5,0,0.0,5,False,NaN,2020-09-23 09:00:14.881983,48,58822.jpg,0.0,0.2,a,1,3,0,...,0,0,0,0,0.0,0.0,0.0,0.0,480,187,470,187,440,175,484,187,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,-0.416667


In [6]:
player_future_fixture.head()

,fixture_id,fixture_code,home_team_id,team_h_score,away_team_id,team_a_score,event,finished,minutes,provisional_start_time,kickoff_time,event_name,is_home,difficulty,player_id,opponent_team_id,team_id
0,44,2128331,12,NaN,1,NaN,5.0,False,0,False,2020-10-17 16:30:00,Gameweek 5,False,5,1,12,1
1,49,2128336,1,NaN,9,NaN,6.0,False,0,False,2020-10-25 19:15:00,Gameweek 6,True,3,1,9,1
2,64,2128351,13,NaN,1,NaN,7.0,False,0,False,2020-11-01 16:30:00,Gameweek 7,False,4,1,13,1
3,69,2128356,1,NaN,2,NaN,8.0,False,0,False,2020-11-07 15:00:00,Gameweek 8,True,3,1,2,1
4,82,2128369,10,NaN,1,NaN,9.0,False,0,False,2020-11-21 15:00:00,Gameweek 9,False,3,1,10,1


In [7]:
df_merge = all_teams[["team_code" , "team_strength" , "team_short_name" , "strength_overall_home" , "strength_overall_away"]]
all_players = all_players.merge(df_merge, on = 'team_code')

df_merge = player_types[['player_type_id' , 'position_name_short']]
all_players = all_players.merge(df_merge , on = "player_type_id")


In [8]:
all_players.head()

,chance_of_playing_next_round,chance_of_playing_this_round,player_code,cost_change_event,cost_change_event_fall,cost_change_start,cost_change_start_fall,dreamteam_count,player_type_id,ep_next,ep_this,event_points,form,player_id,in_dreamteam,news,news_added,now_cost,photo,PPG,selected_by_percent,status,team_id,team_code,total_points,...,creativity,threat,ict_index,influence_rank,influence_rank_type,creativity_rank,creativity_rank_type,threat_rank,threat_rank_type,ict_index_rank,ict_index_rank_type,corners_and_indirect_freekicks_order,corners_and_indirect_freekicks_text,direct_freekicks_order,direct_freekicks_text,penalties_order,penalties_text,PP90,PPMM,VAPM,team_strength,team_short_name,strength_overall_home,strength_overall_away,position_name_short
0,NaN,NaN,37605,0,0,-2,2,0,3,-0.5,0.5,0,0.0,1,False,NaN,NaT,68,37605.jpg,0.0,0.8,a,1,3,0,...,0.0,0.0,0.0,570,233,571,233,570,232,571,233,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,-0.294118,4,ARS,1240,1250,MID
1,75.0,NaN,54694,0,0,-1,1,0,3,1.9,3.5,2,3.0,4,False,Ankle injury - 75% chance of playing,2020-10-08 09:00:12.305506,119,54694.jpg,4.0,22.6,d,1,3,16,...,57.2,103.0,23.8,69,26,60,44,30,13,41,22,NaN,NaN,6.0,NaN,1.0,NaN,4.000000,0.336134,0.168067,4,ARS,1240,1250,MID
2,NaN,NaN,84450,-1,1,-1,1,0,3,0.8,1.8,1,1.3,9,False,NaN,NaT,54,84450.jpg,1.8,0.7,a,1,3,7,...,27.3,21.0,7.9,217,89,130,82,171,94,192,96,NaN,NaN,2.0,NaN,NaN,NaN,2.680851,0.333333,-0.037037,4,ARS,1240,1250,MID
3,NaN,NaN,195735,-1,1,-3,3,0,3,2.2,3.2,6,2.7,18,False,NaN,NaT,77,195735.jpg,2.2,0.9,a,1,3,9,...,20.0,33.0,9.0,195,81,165,100,122,63,167,82,1.0,NaN,3.0,NaN,2.0,NaN,8.526316,0.285714,0.025974,4,ARS,1240,1250,MID
4,0.0,NaN,198849,0,0,-2,2,0,3,0.0,0.5,0,0.0,19,False,Joined Atletico Madrid on loan until the end o...,2020-10-05 22:30:19.674811,48,198849.jpg,0.0,0.2,n,1,3,0,...,0.0,0.0,0.0,483,191,474,192,446,176,487,191,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,-0.416667,4,ARS,1240,1250,MID


## Save to file

In [9]:
all_players.to_csv (r'./data/' + season + '/Merged_data/all_players_merged.csv', index = False, header=True)
